In [ ]:
!pip install tensorflow scikit-learn

In [5]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Parametreler
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50
data_path = "C:/Users/90553/Desktop/mfcc_copd_images"


# Veri artırımı (sade)
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    data_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    data_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation',
    shuffle=False
)

# Sınıf isimleri
class_names = list(train_generator.class_indices.keys())

# Sınıf ağırlıkları
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weight_dict = dict(enumerate(class_weights))

# Model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True  # Baştan eğitilsin

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(len(class_names), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Eğitim
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[early_stop, reduce_lr],
    class_weight=class_weight_dict
)

# Değerlendirme
val_generator.reset()
y_true = val_generator.classes
y_pred_prob = model.predict(val_generator)
y_pred = np.argmax(y_pred_prob, axis=1)

acc = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("\nTest Metrikleri:")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

report = classification_report(y_true, y_pred, target_names=class_names)
print("\nSınıflandırma Raporu:\n")
print(report)


Found 21135 images belonging to 3 classes.
Found 5283 images belonging to 3 classes.


C:\Users\90553\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
661/661 ━━━━━━━━━━━━━━━━━━━━ 1577s 2s/step - accuracy: 0.4806 - loss: 1.0800 - val_accuracy: 0.4333 - val_loss: 1.1043 - learning_rate: 1.0000e-04
Epoch 2/50
661/661 ━━━━━━━━━━━━━━━━━━━━ 1475s 2s/step - accuracy: 0.6643 - loss: 0.7683 - val_accuracy: 0.3786 - val_loss: 1.3714 - learning_rate: 1.0000e-04
Epoch 3/50
661/661 ━━━━━━━━━━━━━━━━━━━━ 1476s 2s/step - accuracy: 0.7557 - loss: 0.5904 - val_accuracy: 0.5188 - val_loss: 1.3020 - learning_rate: 1.0000e-04
Epoch 4/50
661/661 ━━━━━━━━━━━━━━━━━━━━ 1477s 2s/step - accuracy: 0.8225 - loss: 0.4427 - val_accuracy: 0.6633 - val_loss: 0.8210 - learning_rate: 1.0000e-04
Epoch 5/50
661/661 ━━━━━━━━━━━━━━━━━━━━ 1476s 2s/step - accuracy: 0.8722 - loss: 0.3270 - val_accuracy: 0.7289 - val_loss: 0.7412 - learning_rate: 1.0000e-04
Epoch 6/50
661/661 ━━━━━━━━━━━━━━━━━━━━ 1478s 2s/step - accuracy: 0.9094 - loss: 0.2382 - val_accuracy: 0.7963 - val_loss: 0.5458 - learning_rate: 1.0000e-04
Epoch 7/50
661/661 ━━━━━━━━━━━━━━━━━━━━ 1476s 2s/ste